<h2 align="center">Loading Tables; Energy in Colombia from XM's API.</h2>  
By: Jordi Tarragó i Vidal 

Tables previously created in MariaDB will be loaded with data from 2017 to 2022, after this date, data will be updated with a different script.

In [1]:
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
from dgxm import precio_horario
from dgxm import precio_diario

In [3]:
with open('db_string', 'r') as cadena:
    engine = create_engine(cadena.readline())

In [4]:
tables = pd.read_csv('Tablas.csv')

In [5]:
tables.head(10)

,Tabla,Freq,Fecha,Dato,Digitos,Decimales
0,PrecioBolsa,hourly,DATETIME,PrecBolsNaci,12,5
1,PrecioContReg,hourly,DATETIME,PrecPromContRegu,12,5
2,PrecioContNoReg,hourly,DATETIME,PrecPromContNoRegu,12,5
3,PrecioTX1,hourly,DATETIME,PrecBolsNaciTX1,12,5
4,PrecioEscasez,daily,DATE,PrecEsca,12,5
5,PrecioEscasezAct,daily,DATE,PrecEscaAct,12,5
6,VolumenUtilTotal,daily,DATE,VoluUtilDiarEner,18,5
7,VolumenUtilFrac,daily,DATE,PorcVoluUtilDiar,6,5
8,CapacidadUtilTotal,daily,DATE,CapaUtilDiarEner,18,5
9,DemandaSIN,daily,DATE,DemaSIN,18,5


In [6]:
for i in range(len(tables)):
    
    if tables.iloc[i]['Freq'] == 'hourly':
        tmp_df = precio_horario(tables.iloc[i]['Dato'], '2017-01-01', '2022-12-31')
    else:
        tmp_df = precio_diario(tables.iloc[i]['Dato'], '2017-01-01', '2022-12-31')
    
    tmp_df.dropna().to_sql(tables.iloc[i]['Tabla'], engine, index=False, if_exists='append')
    
    print(tables.iloc[i]['Tabla']+':')
    display(tmp_df.describe())
    print(tmp_df.isna().sum())
    
    

PrecioBolsa:


,PrecBolsNaci
count,52560.000000
mean,177.814833
std,107.972218
min,60.312100
25%,95.716590
50%,142.212620
75%,231.238690
max,1035.130680


Fecha            0
PrecBolsNaci    24
dtype: int64
PrecioContReg:


,PrecPromContRegu
count,52560.000000
mean,219.928062
std,35.139398
min,175.002300
25%,189.188243
50%,216.561425
75%,249.684170
max,295.018590


Fecha                0
PrecPromContRegu    24
dtype: int64
PrecioContNoReg:


...

Fecha               0
PorcVoluUtilDiar    1
dtype: int64
CapacidadUtilTotal:


,CapaUtilDiarEner
count,2.190000e+03
mean,1.706324e+10
std,2.350233e+08
min,1.683552e+10
25%,1.692407e+10
50%,1.696933e+10
75%,1.719577e+10
max,1.819418e+10


Fecha               0
CapaUtilDiarEner    1
dtype: int64
DemandaSIN:


,DemaSIN
count,2.190000e+03
mean,1.958685e+08
std,1.508538e+07
min,1.394365e+08
25%,1.868053e+08
50%,1.971079e+08
75%,2.070325e+08
max,2.286212e+08


Fecha      0
DemaSIN    1
dtype: int64


In [7]:
with engine.connect() as conn:
    for i in range(len(tables)):
        tables_temp = pd.read_sql(text('select max(fecha) as "Fecha_Maxima", min(fecha) as "Fecha_Minima" from {};'.format(tables.iloc[i]['Tabla'])), conn)
        print(tables.iloc[i]['Tabla']+': ')
        display(tables_temp)
        tables_temp = pd.read_sql(text('select * from {0} where {1} is null;'.format(tables.iloc[i]['Tabla'],tables.iloc[i]['Dato'])), conn)
        display(tables_temp)
        print('')

PrecioBolsa: 


,Fecha_Maxima,Fecha_Minima
0,2022-12-30 23:00:00,2017-01-01


,Fecha,PrecBolsNaci



PrecioContReg: 


,Fecha_Maxima,Fecha_Minima
0,2022-12-30 23:00:00,2017-01-01


...

,Fecha_Maxima,Fecha_Minima
0,2022-12-30,2017-01-01


,Fecha,CapaUtilDiarEner



DemandaSIN: 


,Fecha_Maxima,Fecha_Minima
0,2022-12-30,2017-01-01


,Fecha,DemaSIN


In [8]:
engine.dispose()